# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

Die Artikel/Wort-Matrix repräsentiert mehrere Dokumente und deren Wörter. Dabei sind die Zeilen die Dokumente und die Spalten alle Wörter, die mindestens einmal in einem der Dokumente auftreten.<br>
Im aktuellen Versuch wird die Artikel/Wort-Matrix durch Nachrichten von RSS-Feeds gebildet. Dabei stellt jede Nachricht ein Dokument in der Matrix dar. In Zelle $d_{i}w_{j}$ steht dann, wie oft Wort $w_{j}$ in Dokument $d_{i}$ vorkommt.

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

Die Multiplikation der Matrix A mit dem Vektor v sieht wie folgt aus:
$A*v=\left( \begin{array}{c}
a_{00} * v_{0} + a_{01} * v_{1} + a_{02} * v_{2} + a_{03} * v_{3} \\
a_{10} * v_{0} + a_{11} * v_{1} + a_{12} * v_{2} + a_{13} * v_{3} \\
a_{20} * v_{0} + a_{21} * v_{1} + a_{22} * v_{2} + a_{23} * v_{3}
\end{array} \right)$

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

Die **Merkmalsmatrix H** definiert, aus welchen Wörtern die Merkmale gebildet werden. Sie ergibt sich aus $m\times c$, mit Merkmalen $m$ und relevanten Wörtern $c$.
Die **Gewichtsmatrix W** definiert, mit welchem Gewicht die einzelenen Merkmale $m$ in den verschiedenen Artikeln $r$ auftreten. Sie ergibt sich aus $m \times r$.

* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

Gegeben seien zwei Numpy Arrays A und B.
- Eine elementweise Multiplikation lässt sich in Numpy mit $C=A*B$ durchführen.
- Eine Matrixmultiplikation lässt sich in Numpy mit np.dot(A, B) durchführen. Dafür muss Matrix B jeoch genauso viele Zeilen haben, wie Matrix A Zeilen hat.

Die Transponierte eines Numpy-Arrays, z.B. Matrix A, lässt sich in Numpy mit np.transpose(A) berechnen.

In [41]:
import feedparser
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from gensim import corpora
import numpy as np
import queue

# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [42]:
feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [43]:
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

In [44]:
def getWordList(doc):
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and
             word not in stopwords.words('english')]
    return words

def getarticlewords(feedList):
    allwords = {}
    articlewords = []
    articletitles = []
    words = []

    for feed in feedList:
        print('*' * 30)
        print(feed)
        f = feedparser.parse(feed)
        for message in f.entries:
            if re.match(r'\S', message.title):
                print(10 * '-' + message.title + 10 * '-')
                articletitles.append(message.title)
                try:
                    fulltext = stripHTML(message.title + ' ' + message.description)
                    words = getWordList(fulltext)
                except:
                    try:
                        fulltext = stripHTML(message.title + ' ' + message.summary)
                        words = getWordList(fulltext)
                    except:
                        print('no description or summary for', message.title)
                docDict = {}
                for word in words:
                    if word in allwords.keys():
                        allwords[word] = allwords[word] + 1
                    else:
                        allwords[word] = 1
                    if word in docDict.keys():
                        docDict[word] = docDict[word] + 1
                    else:
                        docDict[word] = 1
                articlewords.append(docDict)

    nonrelevantwords = [word for word in allwords.keys() if allwords[word] < 4]
    for word in allwords.keys():
        if word not in nonrelevantwords:
            counter = 0
            for a in articlewords:
                if word in a.keys():
                    counter += 1
            if counter / len(articletitles) > 0.3:
                nonrelevantwords.append(word)

    for word in nonrelevantwords:
        catchValue = allwords.pop(word)
        for article in articlewords:
            catchValue = article.pop(word, None)

    for article in articlewords:
        if not article:
            articletitles[articlewords.index(article)] = None
            articlewords[articlewords.index(article)] = None
        # try without this block
        elif len(article) < 3:
            for word in article.keys():
                 allwords[word] -= 1
            articletitles[articlewords.index(article)] = None
            articlewords[articlewords.index(article)] = None

    articlewords = list(filter(None, articlewords))
    articletitles = list(filter(None, articletitles))
    return allwords, articlewords, articletitles

In [45]:
allwords, articlewords, articletitles = getarticlewords(feedlist)

print(50*'*')
for docDict in articlewords:
    print(docDict)

******************************
http://feeds.reuters.com/reuters/topNews
----------China's tariffs on U.S. oil would disrupt $1 billion monthly business----------
----------Democrats, activists rally against Trump's family separation policy----------
----------Syria state media says U.S. bombs Syrian positions but U.S. issues denial----------
----------Audi CEO arrested: Volkswagen----------
----------Magnitude 6.1 quake in Japan's Osaka area kills three, stops factories----------
----------Police search for van that killed one, injured three at Dutch concert----------
----------Yemeni civilians shelter in schools from air strikes in battle for major port----------
----------Colombia president-elect vows to unite nation, alter peace deal----------
----------Google to invest $550 million in Chinese e-commerce giant JD.com----------
----------China's Huawei rebuts Australian security concerns amid Sino-Canberra tensions----------
******************************
http://feeds.reuters.com/reu

******************************
http://rss.cnn.com/rss/edition_world.rss
----------Has the US finally won the fight for Okinawa?----------
----------Elon Musk's Boring Company set to build futuristic transport link in Chicago----------
----------Apple closes law enforcement loophole for the iPhone----------
----------Bitcoin prices have been manipulated, study says----------
----------With all the hacks, is investing in bitcoin safe?----------
----------Experts say this is the food of the future----------
----------When his foster kids arrived with trash bags, he did this----------
----------The family of six that motivates this CNN Hero----------
----------Families slept on hospital floors until one doctor started a shelter----------
----------Goodbye, old iPhone: This could be 40X better----------
----------Bitcoin up 30,000X -- here's your backdoor in----------
----------$19.9 trillion idea hidden in plain sight----------
******************************
http://www.nytimes.com/services

----------DealBook Briefing: China’s Ready for a Trade War. Your Move, Mr. Trump.----------
----------Corner Office: Marc Benioff of Salesforce: ‘Are We Not All Connected?’----------
----------The Getaway: Glamping Slips Into the Mainstream----------
----------Ed Sadlowski, Fiery Steelworkers Insurgent, Dies at 79----------
----------How the I.R.S. Could Punish Trump and His Foundation----------
******************************
http://www.nytimes.com/services/xml/rss/nyt/World.xml
----------Deadly Tensions Rise as India’s Water Supply Runs Dangerously Low----------
----------Did Mexico’s Revelry in World Cup Win Over Germany Cause an Earthquake?----------
----------Hopes for New Era of Malaysian Free Speech Are High, but Pending----------
----------Japan Earthquake Kills at Least 3 and Injures Hundreds More----------
----------Claiming ‘Parallel State’ Cabal, Romania’s Leaders Target Anti-Corruption Prosecutor----------
----------Colombia Elects Iván Duque, Populist Conservative, as Pres

### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

In [46]:
def makematrix(allwords, articlewords):
    dictionary = corpora.Dictionary([allwords.keys()])
    corpus = []
    for doc in articlewords:
        wordList = []
        for k, v in doc.items():
            wordList += [k] * v
        corpus += [dictionary.doc2bow(wordList)]
    matrix = np.zeros(shape=(len(articlewords), len(allwords.keys())))
    for i in range(len(corpus)):
        for tuple in corpus[i]:
            matrix[i][tuple[0]] = tuple[1]
    return matrix, dictionary

In [47]:
matrix, dictionary = makematrix(allwords, articlewords)
print(matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

In [48]:
def cost(A, B):
    result = 0
    if A.shape == B.shape:
        rows = A.shape[0]
        cols = A.shape[1]
        i = 0
        while i < rows:
            j = 0
            while j < cols:
                result += (A[i][j] - B[i][j]) ** 2
                j += 1
            i += 1
        return result
    else:
        print("Error: matrices have different shapes")
        return 0


def nnmf(A, m, it):
    iterations = it
    rows, cols = A.shape
    if m >= cols:
        print("No valid number of features. Must be less than columns.")
        return
    H = np.random.rand(m, cols)
    W = np.random.rand(rows, m)
    costQueue = queue.Queue()
    while it > 0:
        B = np.dot(W, H)
        costs = cost(A, B)
        costQueue.put(costs)
        if it <= iterations - 10:
            if costQueue.get() - costs < 2:
                return H, W
        temp1 = np.array(np.dot(W.transpose(), A))
        temp2 = np.array(np.dot(np.dot(W.transpose(), W), H))
        H = np.array(H * np.true_divide(temp1, temp2))
        W = np.array(W * np.true_divide(np.array(np.dot(A, H.transpose())), np.array(np.dot(np.dot(W, H), H.transpose()))))
        it -= 1
    return H, W

In [49]:
H, W = nnmf(matrix, 20, 200)

## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.



### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.


### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

In [50]:
def showfeatures(H, W, articletitles, wordvec):
    featurelist = []
    for row in H:
        innerList = []
        for key, value in wordvec.items():
            innerList.append((row[value], key))
        innerList.sort(key=lambda tup: tup[0], reverse=True)
        featurelist.append(innerList)
    featurelist = [element[:6] for element in featurelist]

    articlefeatures = []
    for row in W:
        innerList = []
        index = 0
        for index in range(len(featurelist)):
            innerList.append((row[index], index))
        innerList.sort(key=lambda tup: tup[0], reverse=True)
        articlefeatures.append(innerList)
    articlefeatures = [element[:2] for element in articlefeatures]

    for index in range(len(articletitles)):
        print(articletitles[index], articlefeatures[index])
        print('-----------', [x[1] for x in featurelist[articlefeatures[index][0][1]]])
        print('-----------', [x[1] for x in featurelist[articlefeatures[index][1][1]]])
        print(30*'*')

In [51]:
showfeatures(H, W, articletitles, dictionary.token2id)

China's tariffs on U.S. oil would disrupt $1 billion monthly business [(8.698520517833453, 9), (6.345776119007877, 7)]
----------- ['koepka', 'u.s', 'brooks', 'open', 'elizabeth', 'holmes']
----------- ['water', 'last', 'month', 'home', 'low', 'kabul']
******************************
Democrats, activists rally against Trump's family separation policy [(6.250726840074076, 6), (4.955287458740368, 8)]
----------- ['trump', 'president', 'said', 'white', 'report', 'house']
----------- ['new', 'england', 'sharp', 'cancer-causing', 'jersey', 'women']
******************************
Syria state media says U.S. bombs Syrian positions but U.S. issues denial [(26.068421271396975, 19), (0.16003633933220024, 4)]
----------- ['u.s', 'media', 'state', 'military', 'positions', 'syria']
----------- ['arrested', 'said', 'audi', 'volkswagen', 'reuters', 'monday']
******************************
Audi CEO arrested: Volkswagen [(21.25081197304413, 4), (0.05098934341205358, 5)]
----------- ['arrested', 'said', 

Glasgow Jewel Goes Up in Flames for Second Time in 4 Years [(3.8292771673649133, 4), (1.869139988989185, 5)]
----------- ['arrested', 'said', 'audi', 'volkswagen', 'reuters', 'monday']
----------- ['time', 'warner', 'schultz', 'executive', 'howard', 'last']
******************************
‘Are We Still Friends?’ You Speak Out on Trump and Trudeau: The Canada Letter [(5.637478159971454, 6), (3.276887505566694, 16)]
----------- ['trump', 'president', 'said', 'white', 'report', 'house']
----------- ['kim', 'trump', 'north', 'president', 'korean', 'jong-un']
******************************
Trump’s Promises to Kim Jong-un Leave U.S. and Allies Scrambling [(14.841822442922108, 16), (1.9706384849468277, 9)]
----------- ['kim', 'trump', 'north', 'president', 'korean', 'jong-un']
----------- ['koepka', 'u.s', 'brooks', 'open', 'elizabeth', 'holmes']
******************************
China, Paul Manafort, Immigration: Your Friday Evening Briefing [(1.3422404525756413, 18), (1.1332552076827351, 3)]
--

In [52]:
allwords, articlewords, articletitles = getarticlewords(feedlist)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 30, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

******************************
http://feeds.reuters.com/reuters/topNews
----------China's tariffs on U.S. oil would disrupt $1 billion monthly business----------
----------Democrats, activists rally against Trump's family separation policy----------
----------Syria state media says U.S. bombs Syrian positions but U.S. issues denial----------
----------Audi CEO arrested: Volkswagen----------
----------Magnitude 6.1 quake in Japan's Osaka area kills three, stops factories----------
----------Police search for van that killed one, injured three at Dutch concert----------
----------Yemeni civilians shelter in schools from air strikes in battle for major port----------
----------Colombia president-elect vows to unite nation, alter peace deal----------
----------Google to invest $550 million in Chinese e-commerce giant JD.com----------
----------China's Huawei rebuts Australian security concerns amid Sino-Canberra tensions----------
******************************
http://feeds.reuters.com/reu

----------Has the US finally won the fight for Okinawa?----------
----------Elon Musk's Boring Company set to build futuristic transport link in Chicago----------
----------Apple closes law enforcement loophole for the iPhone----------
----------Bitcoin prices have been manipulated, study says----------
----------With all the hacks, is investing in bitcoin safe?----------
----------Experts say this is the food of the future----------
----------When his foster kids arrived with trash bags, he did this----------
----------The family of six that motivates this CNN Hero----------
----------Families slept on hospital floors until one doctor started a shelter----------
----------Goodbye, old iPhone: This could be 40X better----------
----------Bitcoin up 30,000X -- here's your backdoor in----------
----------$19.9 trillion idea hidden in plain sight----------
******************************
http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml
----------Europe Edition: World Cup, Migrant

----------How the I.R.S. Could Punish Trump and His Foundation----------
******************************
http://www.nytimes.com/services/xml/rss/nyt/World.xml
----------Deadly Tensions Rise as India’s Water Supply Runs Dangerously Low----------
----------Did Mexico’s Revelry in World Cup Win Over Germany Cause an Earthquake?----------
----------Hopes for New Era of Malaysian Free Speech Are High, but Pending----------
----------Japan Earthquake Kills at Least 3 and Injures Hundreds More----------
----------Claiming ‘Parallel State’ Cabal, Romania’s Leaders Target Anti-Corruption Prosecutor----------
----------Colombia Elects Iván Duque, Populist Conservative, as President----------
----------Macedonia Signs Historic Deal With Greece on Name Dispute----------
----------Shunned by Italy, Migrants at Sea Arrive in Spain----------
----------As Afghan Cease-Fire Ends, Temporary Friends Hug, Then Return to War----------
----------Plan to Raise a ‘Pious Generation’ Divides Parents in Turkey---

----------- ["japan's", 'three', 'osaka', '6.1', 'magnitude', 'area']
******************************
At Least 17 Dead in Stampede at Graduation Party in Venezuela [(3.773927717926095, 3), (1.1540202345323067, 13)]
----------- ['texas', 'immigrants', 's.u.v', 'sunday', 'dead', 'border']
----------- ['trump', 'people', 'friday', 'jong', 'wants', 'president']
******************************
Battles Rage Around International Airport in Strategic Yemeni City [(2.079437799248196, 9), (1.2055115623113568, 18)]
----------- ['air', 'battle', 'port', 'shelter', 'strikes', 'civilians']
----------- ['trump', 'president', 'donald', 'york', 'new', 'city']
******************************
China’s Official News Media Sharply Criticize Trump [(22.928516770126564, 10), (0.2742020487529599, 27)]
----------- ['chinese', 'media', 'united', 'trump', 'china’s', 'billion']
----------- ['leftist', 'billionaire', 'slim', 'presidential', 'could', 'obrador']
******************************
After a President’s Shockin

In [53]:
def getWordList(doc):
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and word not in stopwords.words('english')]
    words = [PorterStemmer().stem(word) for word in words]
    return words

allwords, articlewords, articletitles = getarticlewords(feedlist)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 20, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

******************************
http://feeds.reuters.com/reuters/topNews
----------China's tariffs on U.S. oil would disrupt $1 billion monthly business----------
----------Democrats, activists rally against Trump's family separation policy----------
----------Syria state media says U.S. bombs Syrian positions but U.S. issues denial----------
----------Audi CEO arrested: Volkswagen----------
----------Magnitude 6.1 quake in Japan's Osaka area kills three, stops factories----------
----------Police search for van that killed one, injured three at Dutch concert----------
----------Yemeni civilians shelter in schools from air strikes in battle for major port----------
----------Colombia president-elect vows to unite nation, alter peace deal----------
----------Google to invest $550 million in Chinese e-commerce giant JD.com----------
----------China's Huawei rebuts Australian security concerns amid Sino-Canberra tensions----------
******************************
http://feeds.reuters.com/reu

----------Separation of parents and children explodes into politically unpredictable test yet for Trump----------
----------The handout parents get as they're separated from their children----------
----------Laura Bush pens scathing column on child separation----------
----------Melania Trump says US should govern 'with heart'----------
----------Angry lawmaker bangs on window and confronts authorities----------
******************************
http://rss.cnn.com/rss/edition_world.rss
----------Has the US finally won the fight for Okinawa?----------
----------Elon Musk's Boring Company set to build futuristic transport link in Chicago----------
----------Apple closes law enforcement loophole for the iPhone----------
----------Bitcoin prices have been manipulated, study says----------
----------With all the hacks, is investing in bitcoin safe?----------
----------Experts say this is the food of the future----------
----------When his foster kids arrived with trash bags, he did this------

----------Wealth Matters: A Guiding Hand From College to Not Just a Job, but a Career----------
----------Why the Fed Tweaked an Obscure Interest Rate This Week----------
----------On the Runway: Is the Independent Designer an Endangered Species?----------
----------DealBook Briefing: China’s Ready for a Trade War. Your Move, Mr. Trump.----------
----------Corner Office: Marc Benioff of Salesforce: ‘Are We Not All Connected?’----------
----------The Getaway: Glamping Slips Into the Mainstream----------
----------Ed Sadlowski, Fiery Steelworkers Insurgent, Dies at 79----------
----------How the I.R.S. Could Punish Trump and His Foundation----------
******************************
http://www.nytimes.com/services/xml/rss/nyt/World.xml
----------Deadly Tensions Rise as India’s Water Supply Runs Dangerously Low----------
----------Did Mexico’s Revelry in World Cup Win Over Germany Cause an Earthquake?----------
----------Hopes for New Era of Malaysian Free Speech Are High, but Pending-------

----------- ['chines', 'media', 'critic', 'unit', 'billion', 'china’']
******************************
The Jobs Recovery: A Longer View [(0.909350381778779, 5), (0.2805055377541126, 15)]
----------- ['england', 'sharp', 'infect', 'cancer-caus', 'women', 'find']
----------- ['execut', 'time', 'last', 'chief', 'year', 'warner']
******************************


In [54]:
def getWordList(doc):
    unrelevantWords = ['reuters', 'cnn', 'nytimes',
                       'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    words = str.split(doc.lower())
    wordsStripped = [word.strip('().,:;!?-"') for word in words]
    words = [word for word in wordsStripped if 2 < len(word) < 20 and word not in stopwords.words('english') and
             word not in unrelevantWords and
             not re.match(r'\$?\d+\€*', word)]
    words = [PorterStemmer().stem(word) for word in words]
    return words

allwords, articlewords, articletitles = getarticlewords(feedlist)
matrix, dictionary = makematrix(allwords, articlewords)
H, W = nnmf(matrix, 20, 200)
showfeatures(H, W, articletitles, dictionary.token2id)

******************************
http://feeds.reuters.com/reuters/topNews
----------China's tariffs on U.S. oil would disrupt $1 billion monthly business----------
----------Democrats, activists rally against Trump's family separation policy----------
----------Syria state media says U.S. bombs Syrian positions but U.S. issues denial----------
----------Audi CEO arrested: Volkswagen----------
----------Magnitude 6.1 quake in Japan's Osaka area kills three, stops factories----------
----------Police search for van that killed one, injured three at Dutch concert----------
----------Yemeni civilians shelter in schools from air strikes in battle for major port----------
----------Colombia president-elect vows to unite nation, alter peace deal----------
----------Google to invest $550 million in Chinese e-commerce giant JD.com----------
----------China's Huawei rebuts Australian security concerns amid Sino-Canberra tensions----------
******************************
http://feeds.reuters.com/reu

----------Melania Trump says US should govern 'with heart'----------
----------Angry lawmaker bangs on window and confronts authorities----------
******************************
http://rss.cnn.com/rss/edition_world.rss
----------Has the US finally won the fight for Okinawa?----------
----------Elon Musk's Boring Company set to build futuristic transport link in Chicago----------
----------Apple closes law enforcement loophole for the iPhone----------
----------Bitcoin prices have been manipulated, study says----------
----------With all the hacks, is investing in bitcoin safe?----------
----------Experts say this is the food of the future----------
----------When his foster kids arrived with trash bags, he did this----------
----------The family of six that motivates this CNN Hero----------
----------Families slept on hospital floors until one doctor started a shelter----------
----------Goodbye, old iPhone: This could be 40X better----------
----------Bitcoin up 30,000X -- here's your 

----------Deadly Tensions Rise as India’s Water Supply Runs Dangerously Low----------
----------Did Mexico’s Revelry in World Cup Win Over Germany Cause an Earthquake?----------
----------Hopes for New Era of Malaysian Free Speech Are High, but Pending----------
----------Japan Earthquake Kills at Least 3 and Injures Hundreds More----------
----------Claiming ‘Parallel State’ Cabal, Romania’s Leaders Target Anti-Corruption Prosecutor----------
----------Colombia Elects Iván Duque, Populist Conservative, as President----------
----------Macedonia Signs Historic Deal With Greece on Name Dispute----------
----------Shunned by Italy, Migrants at Sea Arrive in Spain----------
----------As Afghan Cease-Fire Ends, Temporary Friends Hug, Then Return to War----------
----------Plan to Raise a ‘Pious Generation’ Divides Parents in Turkey----------
----------5 Dead in Texas After S.U.V. Packed With Immigrants Crashes in High-Speed Chase----------
----------Far-Right Protesters Arrested at Kiev Ga

----------- ['macedonia', 'sign', 'greec', 'name', 'deal', 'chang']
----------- ['trump', 'friday', 'peopl', 'want', 'jong', 'kim']
******************************
North Korea Film Glorifies Kim’s World Debut, With Trump in Starring Role [(15.228623436787695, 16), (1.3509499356317252, 15)]
----------- ['kim', 'trump', 'north', 'presid', 'korean', 'leader']
----------- ['world', 'cup', 'germani', 'match', 'earthquak', 'soccer']
******************************
The Week Ahead: Central Bankers Will Debate Policy, and OPEC Meets on Output [(3.8278683717657764, 7), (1.4321322351637842, 8)]
----------- ['trade', 'china', 'u.', 'economi', 'beij', 'tariff']
----------- ['execut', 'chief', 'time', 'last', 'schultz', 'year']
******************************
Feeling Good About the Economy? You’re Probably a Republican [(5.211788520139328, 6), (1.9493209784681411, 14)]
----------- ['trump', 'presid', 'report', 'said', 'separ', 'white']
----------- ['england', 'sharp', 'cancer-caus', 'infect', 'women', 

## Aufgaben

1. Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.
2. Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?
3. Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

In [7]:
#Your markdown